## RNN 创作古诗
在这一章中我们了解到循环神经网络非常擅长处理序列和自然语言处理，文本都是由单词或者汉字按照序列顺序组成的，那么如何能够生成文本呢？下面我们来讲一讲原理，需要你根据这个原理来实现整个网络。

### 原理介绍
前面我们介绍过 RNN 的输入和输出存在多种关系，比如多个输入对一个输出，这个时候输入是一个序列，输出是一个分类结果，就像使用 RNN 做图像分类。

这里我们使用 RNN 来生成文本，网络的输入是一个序列，同时输出也是一个相同长度的序列，结构如下

<img src=https://ws1.sinaimg.cn/large/006tNc79gy1fob5kq3r8jj30mt09dq2r.jpg width=700>

在上面的网络流程中，输入是一个序列 "床 前 明 月 光"，输出也是一个序列 "前 明 月 光 床"。如果你仔细观察可以发现网络的每一步输出都是下一步的输入，这就是其设计思路。

那么对于任意的一段话，比如 "我喜欢小猫"，我们可以将其拆分 "我 喜 欢 小 猫" 这个长度为 5 的序列，网络的每一步输出就是 "喜 欢 小 猫 我"，也就是每个字符的输出就是其**紧跟**的后一个字符。

当然对于一个序列，其最后一个字符后面并没有其他的字符，所以有多种方式选择，比如将序列的第一个字符作为其输出，也就是 "光" 的输出是 "床"，或者将其本身作为输出，也就是 "光" 的输出是 "光"，这里的选择可以有很多，我们使用一种循环的连接，将第一个字符作为最后一个字符的输出。

### 生成文本
这样设计网络的训练流程是为了非常好地生成文本，下面我们说明一下如何进行文本的生成。

首先需要输入网络一段初始的序列进行预热，预热的过程并不需要实际的输出结果，只是为了生成拥有记忆效果的隐藏状态，并将隐藏状态保留下来，接着我们开始正式生成文本，每个字符作为输入都可以得到输出，然后将输出作为下一步的输入，这样就可以不断地生成新的句子，这个过程是可以无限循环下去，或者到达我们的要求输出长度，具体可以看看下面的图示

<img src=https://ws2.sinaimg.cn/large/006tNc79gy1fob5z06w1uj30qh09m0sl.jpg width=800>

讲完了原理之后，下面就该你亲自动手去实现这个网络

In [ ]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import tensorflow as tf

首先我们可以探索一下数据集是什么样的

In [ ]:
with open('./dataset/poetry.txt', 'r') as f:
    poetry_corpus = f.read()

我们取得了前100个字符的结果，其中 `\n` 表示换行符

In [ ]:
poetry_corpus[:100]

In [ ]:
# 看看字符数
print('总的字符数: {}'.format(len(poetry_corpus)))

为了可视化比较方便，我们将换行字符 `\n` 替换成空格

In [ ]:
poetry_corpus = poetry_corpus.replace('\n', ' ').replace('\r', ' ').replace('，', ' ').replace('。', ' ')
print(poetry_corpus[:100])

### 文本数值表示
对于每个文字，电脑并不能像人一样能够有效地识别，所以必须做一个转换，将文字转换成电脑能够识别的数字，相当于每个不同的汉字，都用不同的数字去表示，可以对所有非重复的字符，从 0 开始建立索引

同时可能古诗中会出现一些生僻的字，这些字可能只会出现几次，甚至只会出现一次，引入这些字会增大模型的复杂度，同时也会影响模型的训练，可以将这些词频比较低的字去掉

关于汉字和数字的转换，我们已经为你实现好了一个转换器，感兴趣的同学可以去 `utils.py` 中查看，在之后的练习中，你可以使用这个转换器进行生成文本的转换，下面我们先看看例子

In [ ]:
import numpy as np
from utils import TextConverter

In [ ]:
convert = TextConverter('./dataset/poetry.txt', max_vocab=10000)

上面我们通过数据集建立好了这个转换器 `convert`，下面我们看看如何去调用

In [ ]:
# 得到原始的文本结果
txt_char = poetry_corpus[:11]
print('原始的文本结果: {}'.format(txt_char))
print()

# 通过 convert 将文字转换成数字
num_char = convert.text_to_arr(txt_char)
print('转换成数字之后的结果: {}'.format(num_char))
print()

# 通过 convert 将数字转换成文字
origin_txt_char = convert.arr_to_text(num_char)
print('将数字重新转换成文字: {}'.format(origin_txt_char))

通过上面的例子，你可以看到，能够使用 `convert.text_to_arr` 对一个文本进行数字的转换，通过 `convert.arr_to_text` 将数字转换成文本 

### 构造时序样本数据
对于一整段文本，并不适合全部输入到循环神经网络中，因为我们前面了解到循环神经网络存在着长时依赖的问题，所以需要将整个文本分成很多个序列文本，然后将这些序列文本输入到循环神经网络中进行训练，只要我们定好每个序列的长度，那么序列个数也就被决定了。

In [ ]:
# 每个序列的长度，你可以自行修改
n_step = 20

# 总的序列个数
num_seq = int(len(poetry_corpus) / n_step)

# 去掉最后不足一个序列长度的部分
text = poetry_corpus[:num_seq*n_step]

print('序列的个数: {}'.format(num_seq))

接着需要将序列中所有的文字转换成数字表示，同时重新排列成 **$(num\_seq \times n\_step)$** 的矩阵

完成下面的 `#todo` 的部分

In [ ]:
arr = None #todo: 使用 convert 将文本 text 转换成数字表示的数组
arr = None #todo: 将转换之后的数组重新排列成 (num_seq x n_step) 的形状
arr = arr.astype(np.int32)

In [ ]:
# 不要修改下面的代码
# ================== test =================
if arr.shape == (num_seq, n_step):
    print('Successful!')
else:
    print('Failed!')

据此，我们可以构建 Tensorflow 中的数据读取来训练网络，这里我们将最后一个字符的输出 label 定为输入的第一个字符，也就是"床前明月光"的输出是"前明月光床"，完成下面 #todo 的部分

In [ ]:
class TextDataset(object):
    def __init__(self, arr):
        self.arr = arr
        
    def __getitem__(self, item):
        #TODO: 取得 arr 中的 item 这一个序列
        x = None

        #TODO: 构造上述描述的 label
        y = None

        return x, y
    
    def __len__(self):
        return self.arr.shape[0]

如果你构造好了这个数据集类，我们可以将其实例化

In [ ]:
train_set = TextDataset(arr)

下面我们可以取出其中一个数据集参看一下是否是我们描述的这样，这个数据集需要像上面描述的一样，请自行检查

In [ ]:
x, y = train_set[0]
print('输入的文字序列 x: {}'.format(convert.arr_to_text(x)))
print('输出的文字序列 y: {}'.format(convert.arr_to_text(y)))

### 建立模型
下面我们需要构建这个循环神经网路的网络结构，模型可以定义成非常简单的两层
- 第一层是 RNN 层, **LSTM (GRU)**
- 第二层是线性层，做分类问题，最后输出预测的字符 **slim.fully_connected**

只需要按照提示填写下面的 #todo 部分

- 构造输入

    首先构造一些placeholder作为网络的输入,方便之后代入数据, 需要构建的是:
    - inputs: placeholder, 接收 `[batch_size, n_step]` 的输入, 是输入的词
    - targets: placeholder, 接收 `[batch_size, n_step]` 的输入, 是输入词的对应词, 也就是 label
    - keep_prob: placeholder, 用来表示 dropout 的保留概率

In [ ]:
def build_inputs(batch_size, n_step):
    '''
    
    args:
        batch_size: 一个批次中有多少个序列输入
        num_steps: 一个序列中有多少个词
        
    return:
        inputs: 输入的词
        targets: 输入词的对应词
        keep_prob: dropout 保留概率
    '''
    inputs = None
    targets = None
    keep_prob = None
    
    return inputs, targets, keep_prob

- 构造 RNN

    然后我们开始构造 RNN, 将一个序列中的每个词产生一个输出词. 

    这里我们可以构造一个多层的 RNN, 可以使用 LSTM 或者 GRU 作为 RNN 的基本单元.

In [ ]:
def build_rnn(hidden_size, num_layers, batch_size, keep_prob):
    '''
    
    args:
        keep_prob: dropout 保留概率
        hidden_size: RNN 隐藏层大小
        num_layers: RNN 隐藏层个数
        batch_size: batch size

    return:
        cell: RNN cell
        initial_state: RNN输入时的初始状态
    '''
    
    def build_cell(hidden_size, keep_prob):
        #todo: 得到一个 rnn cell, 可以是 rnn 或者 lstm 或者 gru
        rnn = None
        
        #todo: 添加 dropout
        cell = None
        
        return cell
    
    
    #todo: 得到一个多层的 rnn cell
    cell = None
    
    #todo: 得到 cell 的初始状态
    initial_state = None
    
    return cell, initial_state

- 构造分类层

    现在我们要构造以 RNN 输出的结果为输入, 一个词为输出的全连接层, 这是一个分类问题, 有多少个词就是多少分类

In [ ]:
import tensorflow.contrib.slim as slim

def build_output(rnn_out, in_size, out_size):
    '''
    
    args:
        rnn_out: 上一步rnn的输出
        in_size: rnn输出的特征个数
        out_size: 词的总个数(分类数)
    
    return:
        out: 输出词的概率向量
        logits: softmax之前的结果
    '''

    #todo: rnn_out 的形状是 (batch, n_step, rnn_size), 将形状改成 (batch x n_step, rnn_size)
    # 变成一个2阶矩阵才可以参与到下一步的分类层
    x = None
    
    #todo: 一个全连阶层作为分类层
    logits = None
    
    #todo: softmax 得到概率
    out = None
    
    return out, logits

- 构造损失函数

    这是一个分类问题, 因此我们使用 softmax_with_logits 作为损失函数

In [ ]:
def build_loss(logits, targets, num_classes):
    '''
    
    args:
        logits: softmax之前的结果
        targets: 目标词
        num_classes: 词的总个数(分类数)
    
    return:
        loss: loss tensor.
    '''
    
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    #todo: softmax 分类损失函数
    loss = None
    
    loss = tf.reduce_mean(loss)
    return loss

- 构造训练过程

    接下来我们需要构造训练过程. 前面说到过, RNN 经常会遇到梯度爆炸的问题, 但有一种方法可以避免这类问题, 就是"梯度裁剪".
    
    tensorflow 可以通过**`tf.clip_by_global_norm(tensors, grad_clip)`**函数对`tensors`进行梯度裁剪. 
    
    形象地说, 如果大于`grad_clip`就会将大于的部分剪掉, 这样操作之后, 所有的`tensors`都比`grad_clip`要小, 也就不会存在爆炸的问题了.
    
    因此在这里我们会用到这个方法, 那么我们就不再使用最简单的`optimizer.minimize`这个函数去构造训练过程了, 需要把这个过程拆开, 具体来说分为下面几步:
    
        - 计算所有可训练变量的梯度
        - 对所有梯度进行裁剪
        - 再将梯度应用到原来的变量上去
        
    第一步和第二步应该都知道如何去做, 第三步需要用到一个全新的函数, **`optimizer.apply_gradients`**. 
    
    `optimizer`就是前面我们定义的比如说梯度下降法方法, Momentum方法, Adam方法等等优化器, 每个优化器都有`apply_gradients`方法, 这里不具体展开如何使用这个函数, 大家可以查看下面的函数说明或者参考[这里](https://tensorflow.google.cn/api_docs/python/tf/train/AdamOptimizer#apply_gradients)

---

<img src="https://image.ibb.co/dx7cRn/apply_gradient.png" alt="apply gradient" border="0" />

In [ ]:
def build_optimizer(loss, learning_rate, grad_clip):
    '''
    
    args:
        loss: loss tensor
        learning_rate: 学习率
    
    return:
        optimizer: 优化方法
    '''

    #todo: 获取所有的可训练变量
    tvars = None
    
    #todo: 获取 loss 对 tvars 的梯度
    grads = None
    
    #todo: 使用 tf.clip_by_global_norm 进行梯度裁剪
    grads_clipped = None
    
    #todo: 生成一个 Adam 优化器
    train_op = None
    
    #todo: 使用 apply_gradients 生成一个参数更新 op
    optimizer = None
    
    return optimizer

- 构建完整的 **CharRNN**

In [ ]:
class CharRNN:
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       rnn_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
        '''
        
        args:
            num_classes: 分类数, 也就是字符总数
            batch_size: batch size
            num_steps: 一个序列中出现的字符个数
            rnn_size: rnn 的隐藏层大小
            num_layers: rnn 中的隐藏层个数
            learning_rate: 学习率
            grad_clip: 梯度裁剪常数
            sampling: 是否进行采样
            
        '''
        # 之后我们用这个网络进行 inference 的时候, 我们会传入一个字符进来, 而不是训练时候的
        # 传入 n_step 个字符, 因此在这里用 sampling 来控制
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        #todo: 构建输入
        self.inputs, self.targets, self.keep_prob = None

        #todo: 构建RNN的cell
        cell, self.initial_state = None

        ### 用RNN跑一遍输入得到输出
        # 首先将输入转化成one_hot形式, 相当于给字符编码
        # 这里你也可以使用我们之前讲过的 word_embedding, 将字符嵌入到一个向量里面去
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        #todo: 运行RNN得到输出和最终状态(提示: 使用 tf.nn.dynamic_rnn)
        outputs, state = None
        
        #todo: 将最后的状态保存在 final_state 中
        self.final_state = state
        
        #todo: 得到分类层的结果
        self.prediction, self.logits = None
        
        #todo: 得到损失函数
        self.loss = None
        
        #todo: 得到优化算子
        self.optimizer = None

接下来我们再定义网络的参数

In [ ]:
batch_size = None        # batch_size
rnn_size = None          # rnn中隐藏层的大小
num_layers = None        # rnn中隐藏层的个数
learning_rate = None     # 学习率
keep_prob = None         # dropout保留概率

使用上面的`CharRNN`构造model

In [ ]:
#todo
model = None

In [ ]:
epochs = 100
# Save every N iterations
save_every_n = 1000

下面构造一个读取数据的`generator`, 也可以自行实现

In [ ]:
import random

class build_data_generator:
    def __init__(self, data, batch_size, shuffle=False):
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.nb_of_examples = len(data)
    
    def __call__(self):
        ind = 0
        indices = list(range(self.nb_of_examples))
        
        if self.shuffle:
            random.shuffle(indices)

        while ind + self.batch_size <= self.nb_of_examples:
            x, y = self.data[ind: ind + batch_size]
            ind += batch_size

            yield x, y

        return
    
    def __len__(self):
        return self.nb_of_examples // batch_size

In [ ]:
import time

开始训练

In [ ]:
saver = tf.train.Saver(max_to_keep=50)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    counter = 0
    
    for e in range(epochs):
        #todo: 
        #用 sess 去得到初始 state 保存在 new_state 中
        new_state = None
        
        loss = 0
        dataset = build_data_generator(train_set, batch_size, True)
        
        for x, y in dataset():
            counter += 1
            start = time.time()
            
            #todo:
            # 构造 feed_dict
            # 这里, 我们需要得到model.inputs, model.targets, model.keep_prob, model.initial_state的输入
            # 需要将上一步得到的state作为这一步的model.initial_state
            feed = None
            
            #todo:
            # 使用上面定义的 feed_dict, 运行 session, 获得当前 batch 的 loss, state, 并运行 model.optimizer
            batch_loss, new_state, _ = None
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, rnn_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, rnn_size))

然后我们可以查看在 checkpoints 中所有保存的模型

In [ ]:
tf.train.get_checkpoint_state('checkpoints')

在训练完成之后, 可以从 checkpoints 中恢复模型, 然后我们再给网络输入一个字符, 再让 CharRNN 不断生成新的字符, 也就是让神经网络"写诗"

为了让输出的字符更加丰富随意, 这里我们从模型输出的概率向量中随机选取前几个中的一个作为最后的输出

In [ ]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

现在是时候看看我们训练的神经网络能够写出什么样的诗了, 这里封装成了一个 sample 函数, 需要完成下面的 #todo 部分

注意, 我们在用 CharRNN 写诗的时候, 需要先用输入的字符对网络进行"预热", 这个过程我们不采用网络输出字符的结果, 但是可以得到一个更好的状态, 然后就可以用这个状态作为后续生成新字符的初始状态, 从而获得更好的效果

In [ ]:
def sample(checkpoint, n_samples, rnn_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    
    # 构造CharRNN模型
    # 注意, 这里我们处于测试状态, batch_size 和 n_step 都应为1
    model = CharRNN(convert.vocab_size, rnn_size=rnn_size, sampling=True)
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)

        #todo:
        # 得到初始 state
        new_state = None
        
        # 这一步我们先将输入的几个字符对网络进行"预热", 
        # 这样可以得到更好的 state
        for c in prime:
            x = np.zeros((1, 1))
            x[0, 0] = convert.word_to_int(c)
            
            #todo:
            # 像之前一样设定feed_dict
            feed = None
            
            #todo:
            # 得到概率输出以及当前输出状态
            preds, new_state = None

        c = pick_top_n(preds, convert.vocab_size)
        samples.append(convert.int_to_word(c))

        for i in range(n_samples):
            x[0,0] = c
            
            #todo:
            # 像之前一样设定feed_dict
            feed = None
            
            #todo:
            # 得到概率输出以及当前输出状态
            preds, new_state = None

            c = pick_top_n(preds, convert.vocab_size)
            samples.append(convert.int_to_word(c))
        
    return ''.join(samples)

看一下训练时长最长的模型的效果

In [ ]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, convert.vocab_size, rnn_size, convert.vocab_size, prime="天青色等烟雨")
print(samp)

看一下刚刚开始训练时模型的效果

In [ ]:
checkpoint = 'checkpoints/i1000_l512.ckpt'
samp = sample(checkpoint, convert.vocab_size, rnn_size, convert.vocab_size, prime="天青色等烟雨")
print(samp)

看看模型的中间结果

In [ ]:
checkpoint = 'checkpoints/i10000_l512.ckpt'
samp = sample(checkpoint, convert.vocab_size, rnn_size, convert.vocab_size, prime="天青色等烟雨")
print(samp)

可以看到, 随着模型不断训练, 得到的语句越来越丰富, 越来越完整, 也就是说效果越来越好. 但是本质上这还是一个概率模型, 